Here we prepare the data to be more readible and easy to load for later usage.
Code which takes care of specificity of data from an origin should be done here. Afterwards the data should be more standardized:
* in 1 table (as long as index is same)
* with signal name as column name
* repartitioned by file name
* in parquet format

In [31]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
from tqdm import tqdm_notebook as tqdm

from typing import List
from IPython.display import display
from pathlib import Path

In [32]:
fp = Path('data/UAH-DRIVESET-v1/raw')

You have to ensure that you do not read too much columns. In fact the files have an issue in the sense that there in an extra separator at the end of each line but for some files only. As such the column in each partitions are not uniform as initially loaded by Dask.

In [33]:
def parse_date(x:str):
    return x.str.extract(r'^(\d+)-', expand=False)

def read_data(filepath:Path, col:List[str])->dd.DataFrame:
    return dd.read_csv(
        filepath,
        include_path_column=True,
        sep=' ',
        usecols=list(range(len(col))),
        header=None
    ).map_partitions(lambda x:
        x.iloc[:,:len(col)]\
         .set_axis(col, axis=1)\
         .assign(
            path=x['path'].map(lambda x: str(Path(x).parts[-2])),
            trip=lambda x: x.path.str.extract(r'^\d+-(.+)', expand=False),
            timestamp=lambda x: pd.to_datetime(parse_date(x['path'])) + pd.to_timedelta(x['time'], unit='s')
         )
    )

In [34]:
df_acc = read_data(
    Path(fp, '**/RAW_ACCELEROMETERS.txt'),
    [
        'time',
        'above 50kph',
        'GX',
        'GY',
        'GZ',
        'GX_filt',
        'GY_filt',
        'GZ_filt',
        'Roll',
        'Pitch',
        'Yaw',
])

df_gps = read_data(
    Path(fp, '**/RAW_GPS.txt'),
    [
        'time',
        'Speed',
        'Latitude',
        'Longitude',
        'Altitude',
        'Vertical accuracy',
        'Horizontal accuracy',
        'Course',
        'Difcourse: course variation',
        'Position state',
        'Lanex dist state',
        'Lanex history',
])

df_osm = read_data(
    Path(fp, '**/PROC_OPENSTREETMAP_DATA.txt'),
    [
        'time',
        'Current road maxspeed',
        'Maxspeed reliability',
        'Road type',
        '# of lanes in road',
        'Estimated current lane',
        'Latitude used to query OSM',
        'Longitude used to query OSM',
        'Delay answer OSM query',
        'Speed',
])

In [35]:
display(df_acc.tail())
display(df_gps.tail())
display(df_osm.tail())

,time,above 50kph,GX,GY,GZ,GX_filt,GY_filt,GZ_filt,Roll,Pitch,Yaw,path,trip,timestamp
9047,924.31,0,-0.009,-0.025,-0.017,0.008,-0.018,-0.022,-1.472,-0.184,2.331,20151221120051-26km-D6-AGGRESSIVE-MOTORWAY,26km-D6-AGGRESSIVE-MOTORWAY,2015-12-21 12:16:15.310
9048,924.42,0,-0.023,-0.070,-0.027,-0.015,-0.040,-0.022,-1.471,-0.177,2.322,20151221120051-26km-D6-AGGRESSIVE-MOTORWAY,26km-D6-AGGRESSIVE-MOTORWAY,2015-12-21 12:16:15.420
9049,924.51,0,-0.005,-0.045,-0.028,-0.007,-0.051,-0.026,-1.469,-0.172,2.312,20151221120051-26km-D6-AGGRESSIVE-MOTORWAY,26km-D6-AGGRESSIVE-MOTORWAY,2015-12-21 12:16:15.510
9050,924.61,0,0.003,-0.067,-0.008,-0.011,-0.051,-0.021,-1.470,-0.167,2.306,20151221120051-26km-D6-AGGRESSIVE-MOTORWAY,26km-D6-AGGRESSIVE-MOTORWAY,2015-12-21 12:16:15.610
9051,924.72,0,0.012,-0.063,-0.019,0.011,-0.066,-0.014,-1.469,-0.159,2.298,20151221120051-26km-D6-AGGRESSIVE-MOTORWAY,26km-D6-AGGRESSIVE-MOTORWAY,2015-12-21 12:16:15.720


,time,Speed,Latitude,Longitude,Altitude,Vertical accuracy,Horizontal accuracy,Course,Difcourse: course variation,Position state,Lanex dist state,Lanex history,path,trip,timestamp
910,920.06,59.0,40.505188,-3.354565,595.5,3,5,104.4,3.164,0,1,0,20151221120051-26km-D6-AGGRESSIVE-MOTORWAY,26km-D6-AGGRESSIVE-MOTORWAY,2015-12-21 12:16:11.060
911,921.06,54.5,40.505150,-3.354396,595.5,4,5,107.9,4.922,0,2,0,20151221120051-26km-D6-AGGRESSIVE-MOTORWAY,26km-D6-AGGRESSIVE-MOTORWAY,2015-12-21 12:16:12.060
912,922.08,51.1,40.505104,-3.354241,595.2,4,5,114.6,10.195,0,1,0,20151221120051-26km-D6-AGGRESSIVE-MOTORWAY,26km-D6-AGGRESSIVE-MOTORWAY,2015-12-21 12:16:13.080
913,923.07,48.1,40.505039,-3.354100,595.1,4,5,122.7,0.000,-9,-9,0,20151221120051-26km-D6-AGGRESSIVE-MOTORWAY,26km-D6-AGGRESSIVE-MOTORWAY,2015-12-21 12:16:14.070
914,924.07,44.9,40.504963,-3.353992,595.0,3,5,132.9,0.000,-9,-9,0,20151221120051-26km-D6-AGGRESSIVE-MOTORWAY,26km-D6-AGGRESSIVE-MOTORWAY,2015-12-21 12:16:15.070


,time,Current road maxspeed,Maxspeed reliability,Road type,# of lanes in road,Estimated current lane,Latitude used to query OSM,Longitude used to query OSM,Delay answer OSM query,Speed,path,trip,timestamp
376,916.24,70.0,1,motorway_link,2,1,40.505344,-3.355615,1.193,69.7,20151221120051-26km-D6-AGGRESSIVE-MOTORWAY,26km-D6-AGGRESSIVE-MOTORWAY,2015-12-21 12:16:07.240
377,918.20,70.0,1,motorway_link,2,1,40.505283,-3.355186,1.146,65.7,20151221120051-26km-D6-AGGRESSIVE-MOTORWAY,26km-D6-AGGRESSIVE-MOTORWAY,2015-12-21 12:16:09.200
378,919.79,70.0,1,motorway_link,2,1,40.505219,-3.354764,0.737,59.0,20151221120051-26km-D6-AGGRESSIVE-MOTORWAY,26km-D6-AGGRESSIVE-MOTORWAY,2015-12-21 12:16:10.790
379,921.84,70.0,1,motorway_link,2,1,40.505188,-3.354564,1.791,51.1,20151221120051-26km-D6-AGGRESSIVE-MOTORWAY,26km-D6-AGGRESSIVE-MOTORWAY,2015-12-21 12:16:12.840
380,923.48,70.0,1,motorway_link,2,1,40.505104,-3.354241,1.419,48.1,20151221120051-26km-D6-AGGRESSIVE-MOTORWAY,26km-D6-AGGRESSIVE-MOTORWAY,2015-12-21 12:16:14.480


## Merge (column-on-column)

In [36]:
df = df_gps.merge(df_osm, how="outer", on=['path', 'timestamp'], suffixes=('_gps', '_osm'))
df = df.merge(df_acc, how="outer", on=['path', 'timestamp'], suffixes=('', '_acc'))

df = df.groupby('path').apply(lambda x: x.sort_values('timestamp'))
df = df.reset_index(drop=True)

In [37]:
df['speed'] = df['Speed_gps']
df = df.drop(['Speed_gps', 'Speed_osm'], axis=1)

In [38]:
# display(df_gps['Latitude'].count().compute() - df['Latitude'].count().compute())
# display(df_osm['Current road maxspeed'].value_counts().compute() - df['Current road maxspeed'].value_counts().compute())

In [39]:
with pd.option_context('display.max_columns', None):
    display(df.head())
    display(df.tail())

,time_gps,Latitude,Longitude,Altitude,Vertical accuracy,Horizontal accuracy,Course,Difcourse: course variation,Position state,Lanex dist state,Lanex history,path,trip_gps,timestamp,time_osm,Current road maxspeed,Maxspeed reliability,Road type,# of lanes in road,Estimated current lane,Latitude used to query OSM,Longitude used to query OSM,Delay answer OSM query,trip_osm,time,above 50kph,GX,GY,GZ,GX_filt,GY_filt,GZ_filt,Roll,Pitch,Yaw,trip,speed
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20151111132348-25km-D1-DROWSY-MOTORWAY,NaN,2015-11-11 13:23:48.510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.51,0.0,-0.002,-0.067,0.032,-0.002,-0.065,0.031,-1.675,-0.093,-0.172,25km-D1-DROWSY-MOTORWAY,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20151111132348-25km-D1-DROWSY-MOTORWAY,NaN,2015-11-11 13:23:48.510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.51,0.0,-0.002,-0.067,0.032,-0.002,-0.063,0.030,-1.675,-0.093,-0.172,25km-D1-DROWSY-MOTORWAY,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20151111132348-25km-D1-DROWSY-MOTORWAY,NaN,2015-11-11 13:23:48.510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.51,0.0,0.004,-0.057,0.024,0.000,-0.010,0.005,-1.508,-0.084,-0.154,25km-D1-DROWSY-MOTORWAY,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20151111132348-25km-D1-DROWSY-MOTORWAY,NaN,2015-11-11 13:23:48.510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.51,0.0,-0.002,-0.067,0.032,-0.001,-0.040,0.018,-1.675,-0.093,-0.172,25km-D1-DROWSY-MOTORWAY,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20151111132348-25km-D1-DROWSY-MOTORWAY,NaN,2015-11-11 13:23:48.510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.51,0.0,-0.002,-0.067,0.032,-0.002,-0.056,0.026,-1.675,-0.093,-0.172,25km-D1-DROWSY-MOTORWAY,NaN


,time_gps,Latitude,Longitude,Altitude,Vertical accuracy,Horizontal accuracy,Course,Difcourse: course variation,Position state,Lanex dist state,Lanex history,path,trip_gps,timestamp,time_osm,Current road maxspeed,Maxspeed reliability,Road type,# of lanes in road,Estimated current lane,Latitude used to query OSM,Longitude used to query OSM,Delay answer OSM query,trip_osm,time,above 50kph,GX,GY,GZ,GX_filt,GY_filt,GZ_filt,Roll,Pitch,Yaw,trip,speed


We can see that at the start of the datalogger recording (for the acceleromters) there is the same timestamp for a few points. We'll only take the last values. Same issue is visible in raw data, so it is not a processing mistake.

In [40]:
df = df.groupby('path').apply(
    lambda x: x.drop_duplicates('time', keep='last')
).reset_index(drop=True)

Index in Dask when reset is partition-dependant, as such I created a robust index below

In [41]:
df = df.repartition(partition_size='10MB')

In [42]:
from lib.util import reset_index_dask
df = reset_index_dask(df)

In [43]:
df = df.map_partitions(lambda x: x.rename(str.lower, axis='columns'))

In [44]:
with pd.option_context('display.max_columns', None):
    display(df.head())
    display(df.tail())

,time_gps,latitude,longitude,altitude,vertical accuracy,horizontal accuracy,course,difcourse: course variation,position state,lanex dist state,lanex history,path,trip_gps,timestamp,time_osm,current road maxspeed,maxspeed reliability,road type,# of lanes in road,estimated current lane,latitude used to query osm,longitude used to query osm,delay answer osm query,trip_osm,time,above 50kph,gx,gy,gz,gx_filt,gy_filt,gz_filt,roll,pitch,yaw,trip,speed
idx,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20151111132348-25km-D1-DROWSY-MOTORWAY,NaN,2015-11-11 13:23:48.510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.51,0.0,-0.002,-0.067,0.032,-0.002,-0.065,0.031,-1.675,-0.093,-0.172,25km-D1-DROWSY-MOTORWAY,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20151111132348-25km-D1-DROWSY-MOTORWAY,NaN,2015-11-11 13:23:48.610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.61,0.0,-0.066,-0.100,-0.008,-0.026,-0.079,0.016,-1.675,-0.090,-0.173,25km-D1-DROWSY-MOTORWAY,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20151111132348-25km-D1-DROWSY-MOTORWAY,NaN,2015-11-11 13:23:48.710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.71,0.0,0.008,-0.073,-0.007,-0.025,-0.084,0.000,-1.674,-0.080,-0.175,25km-D1-DROWSY-MOTORWAY,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20151111132348-25km-D1-DROWSY-MOTORWAY,NaN,2015-11-11 13:23:48.810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.81,0.0,-0.007,-0.061,0.004,-0.005,-0.069,-0.001,-1.673,-0.073,-0.176,25km-D1-DROWSY-MOTORWAY,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20151111132348-25km-D1-DROWSY-MOTORWAY,NaN,2015-11-11 13:23:48.920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.92,0.0,0.024,-0.047,0.006,0.002,-0.059,0.005,-1.672,-0.070,-0.178,25km-D1-DROWSY-MOTORWAY,NaN


,time_gps,latitude,longitude,altitude,vertical accuracy,horizontal accuracy,course,difcourse: course variation,position state,lanex dist state,lanex history,path,trip_gps,timestamp,time_osm,current road maxspeed,maxspeed reliability,road type,# of lanes in road,estimated current lane,latitude used to query osm,longitude used to query osm,delay answer osm query,trip_osm,time,above 50kph,gx,gy,gz,gx_filt,gy_filt,gz_filt,roll,pitch,yaw,trip,speed
idx,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
311173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20151111125233-24km-D1-AGGRESSIVE-MOTORWAY,NaN,2015-11-11 13:04:58.450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,745.45,1.0,0.004,-0.009,0.019,-0.003,-0.017,0.009,-1.527,0.010,-2.639,24km-D1-AGGRESSIVE-MOTORWAY,NaN
311174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20151111125233-24km-D1-AGGRESSIVE-MOTORWAY,NaN,2015-11-11 13:04:58.550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,745.55,1.0,-0.013,-0.019,-0.049,0.004,-0.033,-0.005,-1.525,0.015,-2.636,24km-D1-AGGRESSIVE-MOTORWAY,NaN
311175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20151111125233-24km-D1-AGGRESSIVE-MOTORWAY,NaN,2015-11-11 13:04:58.650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,745.65,1.0,-0.082,-0.012,0.001,-0.043,-0.007,-0.024,-1.526,0.014,-2.637,24km-D1-AGGRESSIVE-MOTORWAY,NaN
311176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20151111125233-24km-D1-AGGRESSIVE-MOTORWAY,NaN,2015-11-11 13:04:58.750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,745.75,1.0,-0.041,-0.037,0.025,-0.052,-0.021,0.010,-1.528,0.014,-2.632,24km-D1-AGGRESSIVE-MOTORWAY,NaN
311177,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20151111125233-24km-D1-AGGRESSIVE-MOTORWAY,NaN,2015-11-11 13:04:58.850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,745.85,1.0,-0.019,-0.047,-0.031,-0.032,-0.029,-0.011,-1.526,0.019,-2.630,24km-D1-AGGRESSIVE-MOTORWAY,NaN


In [45]:
df.to_parquet(Path(fp, 'data.parquet'))